In [6]:
import numpy as np
import pandas as pd
from collections import defaultdict
import copy

In [139]:
class DecisionTree:
    def __init__(self,X,y,labels = []):
        self.labels = labels
        self.feature_dict = {}
        self.res = y
        for n in range(len(X[0])):
            self.feature_dict[f"f{n}"] = []
        for input in X:
            for n in range(len(input)):
                self.feature_dict[f"f{n}"].append(str(input[n]))
        self.first_part = Partition(self.feature_dict,self.res)
        self.result = self.first_part.result
    
    def predict(self,newX,show_logic=False):
        #intialize feature dict with test features
        feature_dict = {}
        label_dict = {}
        for n in range(len(newX[0])):
            feature_dict[f"f{n}"] = []
            label_dict[f"f{n}"]=self.labels[n]
        for input in newX:
            for n in range(len(input)):
                feature_dict[f"f{n}"].append(str(input[n]))
        result = []
        for i in range(len(newX)):
            logic = ""
            feat_logic = []
            first_part = self.result
            best_f = self.result[0]
            specific_cat = feature_dict[best_f][i]
            prev_specific_cat = None
            prev_best_feat = best_f
            c_pool = first_part[1][specific_cat]
            print(first_part)
            while not isinstance(c_pool,set):
                #then c_pool is a list [best_f,{cat1:[],cat2:,...}]
                if show_logic:
                    logic+= f"we know {label_dict[best_f]} is {specific_cat}, and "
                    feat_logic.append((best_f,specific_cat))
                prev_best_f = best_f
                #best_f changes after c_pool changes; if c_pool is {result}, then best_f is [best_f,{cat1:{resx},specific_catergory:{result}}]
                best_f = c_pool[0]
                prev_specific_cat = specific_cat
                specific_cat = feature_dict[best_f][i]
                #^ if c_pool is an empty set because the tree did not account for the presence of an unexpected catergory in the best feature:
                if specific_cat in c_pool[1]:
                    c_pool = c_pool[1][specific_cat]
                else:
                    #get the previous catergory we split on and find most common y value present
                    ys = copy.deepcopy(y)
                    for j in reversed(range(len(feature_dict[prev_best_feat]))):
                        if feature_dict[prev_best_feat][j] != prev_specific_cat:
                            ys = np.delete(ys,j)
                    values,counts = np.unique(ys,return_counts = True)
                    c_pool = set([values[np.argmax(counts)]])
            if show_logic:
                logic+= f"we know {label_dict[best_f]} is {specific_cat}, and thus we know {self.labels[len(self.labels)-1]} is {c_pool}"
                feat_logic.append((best_f,specific_cat))
            result.append(list(c_pool))
            print(feat_logic)
            print(logic)
        return result
    

class Partition:
    def __init__(self,feats,y):
        #feats == {'f1':[_, _ ,_...n],'f2':[_,_,_...n],'fk':[_,_,_...n]}
        #y = [_,_,_...n]
        #get feature with highest entropy; if this is the last feature then there is no other partition after
        self.best_feat, self.entropy = Partition.best_feature(feats,y)
        #check if everything in each partition is not the same; if it is then create a partition with values; else create a new partition with one pool of the elemnts on the feature parted and create another one with the other side parted; keep repeating until one feature remains and then use majority of elements in pool to decide
        feat_values = feats[self.best_feat]
        to_ret = defaultdict(set)
        i=0
        for cat in feat_values:
            to_ret[cat].add(y[i])
            i+=1
        #to_ret (for a feature) == {'catergory1':[y1,y2], 'catergory2':[y3,y5], 'catergoryn':[y4]}
        self.result = [self.best_feat,to_ret]
        if len(feats)>1:
            #sort self.result so decisive pools with one element are iterated through first
            #self.result = Partition.sort_dict(self.result)
            for cat in self.result[1]:
                #deleting identified elements from the list so not counted again
                if len(to_ret[cat])==1:
                    continue
                    '''
                    for i in range(len(feats[self.best_feat])):
                        if i>=len(y):
                            break
                        print(f"i: {i} and y:{y}")
                        if feats[self.best_feat][i]==cat and y[i]==list(to_ret[cat])[0]:
                            del y[i]
                            for key in feats:
                                del feats[key][i]
                    '''
                elif len(to_ret[cat])>1:
                    #catergory within feature does not lead to single output; now we must get all values from the others features and outputs that are in the same sample as the catergory and create a new partition based on them (recursive structure)
                    #i.e. if we pick f1 and see that "Red" gives us two possible outputs, then if "Red" is present in n1-n2 idx in f1, then we will take values present in n1-n2 in f0,f2,f3,f4...y
                    #we will then run a partition with these features; the idea is this will keep continuing until either all the features have been used or a single output is produced by a feature (entropy == 0)
                    new_feats  = copy.deepcopy(dict(feats))
                    new_y = list(y)
                    new_feats.pop(self.best_feat)
                    for i in reversed(range(len(feats[self.best_feat]))):
                        if feats[self.best_feat][i] != cat:
                            for key in new_feats:
                                del new_feats[key][i]
                            del new_y[i]
                    #cant just use to_ret[cat] as new_y because it is a set and will not contain all y values
                    to_ret[cat]=Partition(new_feats,new_y).result

    @classmethod
    def sort_dict(dict):
        dict_to_ret = defaultdict(set)
        keys= dict.keys()
        while keys != None:
            min_key = keys[0]
            min_length = len((dict[keys[0]]))
            for key in keys:
                if len(dict[key])<min_length:
                    min_length = len(dict[key])
                    min_key = key
            dict_to_ret[keys.pop(min_key)] = dict[min_key]
            
    @classmethod
    def best_feature(cls, feats,y,show_entropies = False):
        lowest_entropy = 1
        best_feat = None
        entropies = []
        for feat in feats:
            feat_arr = feats[feat]
            temp = defaultdict(list)
            i=0
            for cat in feat_arr:
                temp[cat].append(y[i])
                i+=1
            curr_entropy = Partition.get_conditional_entropy(temp,len(y))
            if curr_entropy<lowest_entropy:
                lowest_entropy = curr_entropy
                best_feat = feat
            entropies.append(curr_entropy)
        if show_entropies:
            print(entropies)
        return feat,lowest_entropy
    @classmethod
    def get_conditional_entropy(cls,dict,size):
        t_entropy = 0
        for cat in dict:
            entropy = 0
            _,counts = np.unique(dict[cat],return_counts = True)
            t_count = len(dict[cat])
            for count in counts:
                entropy += (count/t_count)*np.emath.logn(2,t_count/count)
            t_entropy += (len(dict[cat])/size)*entropy
        return t_entropy
            
            

In [8]:
#test data
import pandas as pd

In [16]:
df = pd.read_csv('Efficiency.csv')
df.head()

,Motor,Wheels,Door,Size,Efficiency
0,no,2,0,small,good
1,no,3,0,small,bad
2,yes,2,0,small,good
3,yes,4,2,small,bad
4,yes,4,3,medium,good


In [10]:
X = np.array(df.drop(['Efficiency'],axis = 1))
X

array([['no', 2, 0, 'small'],
       ['no', 3, 0, 'small'],
       ['yes', 2, 0, 'small'],
       ['yes', 4, 2, 'small'],
       ['yes', 4, 3, 'medium'],
       ['yes', 4, 4, 'medium'],
       ['yes', 4, 4, 'large']], dtype=object)

In [11]:
y = np.array(df['Efficiency'])
y

array(['good', 'bad', 'good', 'bad', 'good', 'good', 'bad'], dtype=object)

In [140]:
d = DecisionTree(X,y,labels = ["Motor","Wheels","Door","Size","Efficiency"])

In [141]:
d.predict([['yes','4','3','large','good'],['yes','3','0','small','good']],show_logic=True)

['f3', defaultdict(<class 'set'>, {'small': ['f2', defaultdict(<class 'set'>, {'0': ['f1', defaultdict(<class 'set'>, {'2': {'good'}, '3': {'bad'}})], '2': {'bad'}})], 'medium': {'good'}, 'large': {'bad'}})]
[('f3', 'large')]
we know Size is large, and thus we know Efficiency is {'bad'}
['f3', defaultdict(<class 'set'>, {'small': ['f2', defaultdict(<class 'set'>, {'0': ['f1', defaultdict(<class 'set'>, {'2': {'good'}, '3': {'bad'}})], '2': {'bad'}})], 'medium': {'good'}, 'large': {'bad'}})]
[('f3', 'small'), ('f2', '0'), ('f1', '3')]
we know Size is small, and we know Door is 0, and we know Wheels is 3, and thus we know Efficiency is {'bad'}


[['bad'], ['bad']]

In [130]:
y_predict = d.predict(X,show+)
y_predict

['f3', defaultdict(<class 'set'>, {'small': ['f2', defaultdict(<class 'set'>, {'0': ['f1', defaultdict(<class 'set'>, {'2': {'good'}, '3': {'bad'}})], '2': {'bad'}})], 'medium': {'good'}, 'large': {'bad'}})]
we know Size is small, and we know Door is 0, and we know Wheels is 2, and thus we know Efficiency is {'good'}
['f3', defaultdict(<class 'set'>, {'small': ['f2', defaultdict(<class 'set'>, {'0': ['f1', defaultdict(<class 'set'>, {'2': {'good'}, '3': {'bad'}})], '2': {'bad'}})], 'medium': {'good'}, 'large': {'bad'}})]
we know Size is small, and we know Door is 0, and we know Wheels is 3, and thus we know Efficiency is {'bad'}
['f3', defaultdict(<class 'set'>, {'small': ['f2', defaultdict(<class 'set'>, {'0': ['f1', defaultdict(<class 'set'>, {'2': {'good'}, '3': {'bad'}})], '2': {'bad'}})], 'medium': {'good'}, 'large': {'bad'}})]
we know Size is small, and we know Door is 0, and we know Wheels is 2, and thus we know Efficiency is {'good'}
['f3', defaultdict(<class 'set'>, {'small': 

[['good'], ['bad'], ['good'], ['bad'], ['good'], ['good'], ['bad']]

In [42]:
y_predict=np.reshape(y_predict,[7,])
y=np.reshape(y,[7,])

In [44]:
score = np.mean(np.equal(y_predict,y)) * 100
score

100.0